In [1]:
import tensorwrap as tf
tf.__version__

'0.0.0.3'

In [2]:
from tensorwrap import nn

In [3]:
layer = tf.nn.layers.Dense(units = 10)
layer1 = tf.nn.layers.Dense(units = 1)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
x = tf.range(1, 101, dtype=tf.float32)
x /= 100
x = tf. expand_dims(x, axis = 1)

In [5]:
model = nn.Sequential([
    layer1,
    layer
])

In [6]:
model.compile(
    loss = tf.nn.losses.mse,
    optimizer = tf.nn.optimizers.SGD(),
    metrics = tf.nn.losses.mae
)

In [7]:
%timeit model.fit(x, x + 10, epochs = 100)

Epoch 1|100 
[=========================]    Loss:  106.29213     Metric:   10.30689
Epoch 2|100 
[=========================]    Loss:   96.28480     Metric:    9.80841
Epoch 3|100 
[=========================]    Loss:   85.05566     Metric:    9.21526
Epoch 4|100 
[=========================]    Loss:   73.37552     Metric:    8.55245
Epoch 5|100 
[=========================]    Loss:   61.92989     Metric:    7.84583
Epoch 6|100 
[=========================]    Loss:   51.25510     Metric:    7.12014
Epoch 7|100 
[=========================]    Loss:   41.71068     Metric:    6.39742
Epoch 8|100 
[=========================]    Loss:   33.48404     Metric:    5.69601
Epoch 9|100 
[=========================]    Loss:   26.61771     Metric:    5.03002
Epoch 10|100 
[=========================]    Loss:   21.04746     Metric:    4.40943
Epoch 11|100 
[=========================]    Loss:   16.64166     Metric:    3.84035
Epoch 12|100 
[=========================]    Loss:   13.23528     Metric: 

In [8]:
model.evaluate(x, x + 10)

Epoch 1|1 
[=========================]    Loss:    4.46778     Metric:    1.73676


In [9]:
class Dense(nn.layers.Layer):
    def __init__(self, units) -> None:
        super().__init__() # Needed for making it JIT compatible.
        self.units = units # Defining the output shape.
        self.built = False
  
    def build(self, input_shape: tuple) -> None:
        self.kernel = self.add_weights([input_shape[-1], self.units],
                                       initializer = 'glorot_uniform')
        self.bias = self.add_weights([self.units],
                                     initializer = 'zeros')
        super().build(input_shape) # Needed to check dimensions and build.
    
    # Use call not __call__ to define the flow. No tf.function needed either.
    def call(self, inputs):
        return inputs @ self.kernel + self.bias

In [10]:
layer = Dense(units = 2)

In [11]:
layer(x)

Array([[0.00453204, 0.00516343],
       [0.00906409, 0.01032686],
       [0.01359613, 0.01549029],
       [0.01812818, 0.02065372],
       [0.02266022, 0.02581715],
       [0.02719226, 0.03098058],
       [0.03172431, 0.03614401],
       [0.03625635, 0.04130744],
       [0.0407884 , 0.04647087],
       [0.04532044, 0.0516343 ],
       [0.04985248, 0.05679773],
       [0.05438453, 0.06196116],
       [0.05891657, 0.06712459],
       [0.06344862, 0.07228802],
       [0.06798066, 0.07745145],
       [0.0725127 , 0.08261488],
       [0.07704475, 0.08777831],
       [0.08157679, 0.09294175],
       [0.08610883, 0.09810517],
       [0.09064088, 0.1032686 ],
       [0.09517292, 0.10843202],
       [0.09970497, 0.11359546],
       [0.10423701, 0.11875889],
       [0.10876905, 0.12392232],
       [0.1133011 , 0.12908575],
       [0.11783314, 0.13424918],
       [0.12236519, 0.13941261],
       [0.12689723, 0.14457604],
       [0.13142927, 0.14973946],
       [0.13596132, 0.1549029 ],
       [0.

In [12]:
tf.save_model(model, filepath="model.h5")

In [13]:
model = tf.load_model(filepath="model.h5")